In [2]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

model = AzureChatOpenAI(model="gpt-4o",
                        api_version="2025-04-01-preview",
                        temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요. 제 이름은 안민재입니다."),
    AIMessage("안녕하세요. 안민재님. 반가워요."),
    HumanMessage("제 이름이 뭐에요?")
]

ai_message = model.invoke(messages)

print(ai_message.content)

당신의 이름은 안민재입니다! 😊


In [3]:
# Streaming

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕!"),
]
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

안녕하세요! 😊 어떻게 도와드릴까요?

In [6]:
# PromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
                                      다음 요리의 레시피를 생각해주세요.
                                      요리명: {dish}
                                      """)

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value.text)


                                      다음 요리의 레시피를 생각해주세요.
                                      요리명: 카레
                                      


In [ ]:
# ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
        ("ai", "{recipe}"), #("assistant", "") 도 가능
    ]
)
prompt_value = prompt.invoke({"dish": "카레", "recipe": "카레 레시피"})
pprint(prompt_value.messages)

[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='카레', additional_kwargs={}, response_metadata={}),
 AIMessage(content='카레 레시피', additional_kwargs={}, response_metadata={})]


In [20]:
# Messages Placeholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요, 저는 안민재입니다.!"),
            AIMessage(content="안녕하세요, 안민재님. 반가워요."),
        ],
        "input": "제 이름이 뭐에요?",
    }
)

pprint(prompt_value.messages)

[SystemMessage(content='You are helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕하세요, 저는 안민재입니다.!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 안민재님. 반가워요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐에요?', additional_kwargs={}, response_metadata={})]


In [21]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

당신의 이름은 안민재입니다! 😊


In [ ]:
# Output Parsers
# PydanticOutputParser

from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")



from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [23]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "사용자가 입력한 요리의 레시피를 생각해 주세요. \n\n"
         "{format_instructions}"
         ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})

print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리의 레시피를 생각해 주세요. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [26]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

ai_message = model.invoke(prompt_value)
print(ai_message.content)


```json
{
  "ingredients": [
    "카레 가루",
    "감자",
    "당근",
    "양파",
    "고기 (소고기 또는 닭고기)",
    "물",
    "식용유",
    "소금",
    "후추"
  ],
  "steps": [
    "감자, 당근, 양파를 먹기 좋은 크기로 썬다.",
    "고기를 적당한 크기로 썰고 소금과 후추로 간을 한다.",
    "냄비에 식용유를 두르고 양파를 볶아 향을 낸다.",
    "고기를 넣고 겉면이 익을 때까지 볶는다.",
    "감자와 당근을 넣고 함께 볶는다.",
    "물을 냄비에 붓고 끓인다.",
    "재료가 익으면 카레 가루를 넣고 잘 섞는다.",
    "약한 불에서 카레가 걸쭉해질 때까지 끓인다.",
    "완성된 카레를 밥과 함께 제공한다."
  ]
}
```


In [27]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)


<class '__main__.Recipe'>
ingredients=['카레 가루', '감자', '당근', '양파', '고기 (소고기 또는 닭고기)', '물', '식용유', '소금', '후추'] steps=['감자, 당근, 양파를 먹기 좋은 크기로 썬다.', '고기를 적당한 크기로 썰고 소금과 후추로 간을 한다.', '냄비에 식용유를 두르고 양파를 볶아 향을 낸다.', '고기를 넣고 겉면이 익을 때까지 볶는다.', '감자와 당근을 넣고 함께 볶는다.', '물을 냄비에 붓고 끓인다.', '재료가 익으면 카레 가루를 넣고 잘 섞는다.', '약한 불에서 카레가 걸쭉해질 때까지 끓인다.', '완성된 카레를 밥과 함께 제공한다.']


In [29]:
# StrOutputParser
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요, 저는 AI 어시턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요, 저는 AI 어시턴트입니다.


In [30]:
# LCEL (LangChain Expression Language)
# 연쇄적으로 연결하고 싶은 경우
# Prompt template 을 채우고, 그 결과를 Chat model에 제공한 후 그 결과를 Python 객체로 변환

# prompt와 model 연결
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}")
    ]
)

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

chain = prompt | model

ai_message = chain.invoke({"dish": "카레"})
print(ai_message.content)


카레는 간단하면서도 맛있게 즐길 수 있는 요리입니다. 아래는 기본적인 한국식 카레 레시피입니다. 

### 재료:
- **카레 가루**: 1봉지 (약 100g, 인원수에 따라 조절)
- **고기**: 돼지고기, 소고기, 닭고기 중 선택 (약 200~300g)
- **감자**: 2~3개
- **당근**: 1개
- **양파**: 1~2개
- **물**: 약 800ml (카레 가루 포장지에 적힌 양을 참고)
- **식용유**: 약간
- **밥**: 적당량 (카레와 함께 곁들일 밥)

---

### 조리 방법:

1. **재료 손질**:
   - 감자, 당근, 양파는 껍질을 벗기고 먹기 좋은 크기로 깍둑썰기합니다.
   - 고기는 한입 크기로 썰어 준비합니다.

2. **재료 볶기**:
   - 냄비에 식용유를 두르고 중불로 가열합니다.
   - 고기를 먼저 넣고 겉면이 익을 때까지 볶습니다.
   - 고기가 익으면 감자, 당근, 양파를 넣고 함께 볶아줍니다. (채소가 살짝 투명해질 때까지)

3. **물 넣기**:
   - 볶은 재료에 물을 붓고 끓입니다. 물의 양은 카레 가루 포장지에 적힌 대로 조절하세요.
   - 센 불에서 끓이다가 끓기 시작하면 중불로 줄이고 감자가 익을 때까지 약 10~15분 정도 끓입니다.

4. **카레 가루 넣기**:
   - 감자가 익으면 불을 약하게 줄이고 카레 가루를 넣습니다.
   - 카레 가루가 뭉치지 않도록 잘 저어가며 섞어줍니다.

5. **마무리**:
   - 카레가 걸쭉해질 때까지 약 5분 정도 더 끓입니다.
   - 간을 보고 부족한 경우 소금이나 후추로 조절합니다.

6. **완성**:
   - 따뜻한 밥 위에 카레를 얹어 내놓습니다. 기호에 따라 김치, 피클, 계란후라이 등을 곁들여도 좋습니다.

---

### 팁:
- 더 풍미를 원한다면 사과, 파인애플, 버섯 등을 추가해도 좋습니다.
- 매운맛을 좋아한다면 고춧가루나 청양고추를 약간 넣어보세요.
- 카레는 하루 정도 숙성시키면 맛이 더 깊어집니다.

맛있게 

In [ ]:
# StrOutputParser를 연결에 추가
